# DLA Abund Cookbook (v1.0)

In [1]:
# imports
# suppress warnings for these examples
import warnings
warnings.filterwarnings('ignore')

import json, io
import numpy as np

import astropy.units as u

from linetools.isgm.abscomponent import AbsComponent
from linetools.lists.linelist import LineList
from linetools.spectra.io import readspec
import linetools
from linetools.abund.relabund import RelAbund

from pyigm.abssys.dla import DLASystem

## 0.  Analyze NHI, Continuum fit the spectrum (or spectra)
   
These steps are not described here.  

For continuum fitting,  we recommend lt_continuum from linetools

    lt_continuum spec.fits

## 1.  Generate a simple DLA system

Requires $z$, $N_{\rm HI}$ and the coordinates

$v_{\rm lim}$ defaults to 500 km/s if input as None

In [2]:
# PH957
vlim = None
dla = DLASystem(('01:03:11.38','+13:16:16.7'), 2.309, vlim, 21.37, sig_NHI=[0.08,0.08])
dla

<DLASystem: 01:03:11.38 13:16:16.7, 2.309, NHI=21.37, Z/H=0>

In [3]:
# Write
dla.write_json()

Wrote J010311.38+131616.7_z2.309 system to J010311.38+131616.7_z2.309.json file


## 2. AbsKin GUI for Line IDs

   * Requires xastropy for now
   * Best run off the JSON file
   * And from the command line
   
   lt_xabssys PH957_f.fits J010311.38+131616.7_z2.309.json -outfile J010311.38+131616.7_z2.309_HIRES.json
   
   
   
   * User modifies the velocity limits
   * Identifies blends
   * Sets upper/lower limits as desired
   * Rejects lines that are not worth analysis

In [4]:
# Load
json_fil = '../../pyigm/abssys/tests/files/J010311.38+131616.7_z2.309_ESI.json'
dla = DLASystem.from_json(json_fil)

## 3. Measure EWs and AODM columns
    

In [5]:
# Load spectrum from linetools
spec_fil = linetools.__path__[0]+'/spectra/tests/files/PH957_f.fits'
spec = readspec(spec_fil)

In [6]:
# EW
dla.measure_restew(spec=spec)

In [7]:
# AODM
dla.measure_aodm(spec=spec)

You may now wish to update the component column densities with update_component_colm()


In [8]:
# View
dla.fill_trans()
dla._trans

wrest,name,Z,ion,Ej,z,EW,sig_EW,flag_N,logN,sig_logN
Angstrom,,,,1 / cm,,Angstrom,Angstrom,,,
float64,str10,int64,int64,float64,float64,float64,float64,int64,float64,float64
1656.9284,CI 1656,6,1,0.0,2.309,-0.0153520621237,0.00547148105662,3,0.0,0.154038431267
1334.5323,CII 1334,6,2,0.0,2.309,0.477547920281,0.00431757861874,2,14.7052705466,0.0164072954838
1335.7077,CII* 1335,6,2,63.42,2.309,0.134848109253,0.004443903355,1,13.9177304477,0.015082932929
1548.195,CIV 1548,6,4,0.0,2.309,0.0544229111256,0.00557312487645,1,13.1525999956,0.0440273760351
1550.77,CIV 1550,6,4,0.0,2.309,0.0296155887117,0.00537307146159,1,13.175445022,0.0786629498623
1302.1685,OI 1302,8,1,0.0,2.309,0.555291486242,0.00428915442887,2,15.2203065348,0.022034214781
2852.9642,MgI 2852,12,1,0.0,2.309,0.254069422268,0.0154674371474,1,12.4243209562,0.0253720541715
2796.3543,MgII 2796,12,2,0.0,2.309,0.948237352007,0.00959382579309,2,13.691383214,0.0120288970248


## 4. Update Components

In [9]:
dla.update_component_colm()

In [10]:
dla.fill_ionN()

In [11]:
dla._ionN

Z,ion,A,Ej,z,vmin,vmax,flag_N,logN,sig_logN
,,,1 / cm,,km / s,km / s,,,
int64,int64,int64,float64,float64,float64,float64,int64,float64,float64
6,1,0,0.0,2.309,-206.600206903,-44.7222752434,3,0.0,0.154038431267
6,2,0,0.0,2.309,-206.600206903,40.8934449356,2,14.7052705466,0.0164072954838
6,2,0,63.42,2.309,-206.600206903,-10.9257491888,1,13.9177304477,0.015082932929
6,4,0,0.0,2.309,-206.600206903,40.8934449356,1,13.1577289847,0.0384285096556
8,1,0,0.0,2.309,-158.105896737,40.8934449356,2,15.2203065348,0.022034214781
12,1,0,0.0,2.309,-206.600206903,-56.5413246803,1,12.4243209562,0.0253720541715
12,2,0,0.0,2.309,-206.600206903,40.8934449356,2,13.9478760668,0.00871183443821
13,2,0,0.0,2.309,-206.600206903,40.8934449356,1,13.207256792,0.00591161845065


## 5. Relative Abundances (relative to Solar)

In [12]:
dla.XY = RelAbund.from_ionclm_table((1,dla.NHI, dla.sig_NHI[0]), dla._ionN)

Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


In [13]:
dla.XY.elements['H'].name

u'Hydrogen'

In [14]:
dla.XY.table()

Z,Name,flag,[X/H],sig([X/H])
int64,unicode2,int64,float64,float64
6,C,2,-3.09472945341,0.0164072954838
8,O,2,-2.83969346524,0.022034214781
12,Mg,2,-2.95212393322,0.00871183443821
13,Al,1,-2.59274320801,0.00591161845065
14,Si,1,-1.56413074476,0.0255411728835
26,Fe,1,-2.49692737255,0.00318602578944
28,Ni,1,-1.54644182533,0.0204659988589
30,Zn,1,-1.46240425503,0.0599154533309


In [15]:
dla.XY.table('Fe')

Z,Name,flag,[X/Fe],sig([X/Fe])
int64,unicode2,int64,float64,float64
6,C,2,-0.59780208086,0.0167137699345
8,O,2,-0.342766092684,0.0222633641066
12,Mg,2,-0.455196560666,0.0092761424962
13,Al,1,-0.0958158354613,0.00671550393024
14,Si,1,0.93279662779,0.0257391194993
28,Ni,1,0.950485547222,0.0207125051509
30,Zn,1,1.03452311752,0.0600001025681
